In [1]:
import numpy as np
import os
import cPickle as pkl
import scipy.signal
import numpy.fft as fft
import sys
import optparse
from PIL import Image
import re
import itertools
from scipy import ndimage

import time
import datetime

import pandas as pd

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure
import csv

import pymworks 
import pandas as pd
import operator
import codecs

import scipy.io
import copy


# Abstract struct class       
class Struct:
    def __init__ (self, *argv, **argd):
        if len(argd):
            # Update by dictionary
            self.__dict__.update (argd)
        else:
            # Update by position
            attrs = filter (lambda x: x[0:2] != "__", dir(self))
            for n in range(len(argv)):
                setattr(self, attrs[n], argv[n])


class cycstruct(Struct):
    times = []
    idxs = 0
    vals = 0
    ordernum = 0
    triggers = 0

def get_timekey(item):
    return item.time


In [2]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
def get_session_bounds(dfn):
    df = None
    df = pymworks.open(dfn)                                                          # Open the datafile

    # First, find experiment start, stop, or pause events:
    modes = df.get_events('#state_system_mode')                                      # Find timestamps for run-time start and end (2=run)
    start_ev = [i for i in modes if i['value']==2][0]                                # 2=running, 0 or 1 = stopped

    run_idxs = [i for i,e in enumerate(modes) if e['time']>start_ev['time']]         # Get all "run states" if more than 1 found

    end_ev = next(i for i in modes[run_idxs[0]:] if i['value']==0 or i['value']==1)  # Find the first "stop" event after the first "run" event

    # Create a list of runs using start/stop-event times (so long as "Stop" button was not pressed during acquisition, only 1 chunk of time)
    bounds = []
    bounds.append([start_ev.time, end_ev.time])
    for r in run_idxs[1:]: 
        if modes[r].time < end_ev.time:  # Ignore any extra "run" events if there was no actual "stop" event
            continue
        else:                            # Otherwise, find the next "stop" event if any additional/new "run" events found.
            try:
                stop_ev = next(i for i in modes[r:] if i['value']==0 or i['value']==1)
            except StopIteration:
                end_event_name = 'trial_end'
                print "NO STOP DETECTED IN STATE MODES. Using alternative timestamp: %s." % end_event_name
                stop_ev = df.get_events(end_event_name)[-1]
                print stop_ev
            bounds.append([modes[r]['time'], stop_ev['time']])

    bounds[:] = [x for x in bounds if ((x[1]-x[0])/1E6)>1]
    # print "................................................................"
    print "****************************************************************"
    print "Parsing file\n%s... " % dfn
    print "Found %i start events in session." % len(bounds)
    print "Bounds: ", bounds
    for bidx, bound in enumerate(bounds):
        print "bound ID:", bidx, (bound[1]-bound[0])/1E6, "sec"
    print "****************************************************************"

    return df, bounds

In [121]:
def get_image_events(df, boundary, pixelclock_evs=[], stimtype='grating', mask=False):
    
    # Get all stimulus-udpate events within bounds:
    tmp_display_evs = df.get_events('#stimDisplayUpdate')                                                  # Get all stimulus-display-update events
    display_evs = [e for e in tmp_display_evs if e.value and not e.value[0]==None]                         # Filter out empty display-update events
    display_evs = [d for d in display_evs if d.time <= boundary[1] and d.time >= boundary[0]]              # Only include display-update events within time boundary of the session

    if len(pixelclock_evs)>0:
        pixelclock_evs = [i for i in display_evs for v in i.value if 'bit_code' in v.keys()]
        num_non_stimuli = 3 # N stimuli on screen: pixel clock, background, image
    if len(pixelclock_evs)==0:
        print "No pixel clock."
        pixelclock_evs = display_evs
        num_non_stimuli = 2 # N stimuli on screen: background, image
       
    # Get stimulus-onset info parsed into trials:
    if stimtype=='image':
        tmp_image_evs = [d for d in pixelclock_evs for i in d.value if 'filename' in i.keys() and '.png' in i['filename']]
        #stimevents.append(imdevs)

        # Find blank ITIs:
        if mask is True:
            iti_evs = [i for i in pixelclock_evs for v in i.value if v['name']=='blue_mask' and i not in tmp_image_evs]
        else:
            iti_evs = [i for i in pixelclock_evs if i.time>image_evs[0].time and i not in tmp_image_evs]

        tmp_trial_evs = tmp_image_evs + iti_evs
        trial_evs = sorted(tmp_trial_evs, key=get_timekey)
        
        image_evs = tmp_image_evs

    elif stimtype=='grating':
        #tmp_image_evs = [d for d in display_evs for i in d.value if i['name']=='gabor']
        tmp_image_evs = [d for d in display_evs for i in d.value if i['type']=='drifting_grating']
        
        start_times = [i.value[1]['start_time'] for i in tmp_image_evs] # Use start_time to ignore dynamic pixel-code of drifting grating since stim as actually static
        find_static = np.where(np.diff(start_times) > 0)[0]
        find_static = np.append(find_static, 0)
        find_static = sorted(find_static)
        image_evs = [tmp_image_evs[i+1] for i in find_static]
        print "got image events"
        #stimevents.append(imtrials)
        
        # Make sure only the 1st stimulus after a new-stim flag is counted (for guarantee-reward) experiments:
        newstim_evs = df.get_events('new_stimulus')
        new_stim_evs = [i for i in newstim_evs if i.value==1]
        print "N new-stimulus events:", len(new_stim_evs)
        first_image_idxs = []
        for idx,newev in enumerate(new_stim_evs[0:-1]):
            possible_image_evs = [i for i,ev in enumerate(image_evs) if ev.time>newev.time and ev.time<new_stim_evs[idx+1].time]
            first_image_idxs.append(possible_image_evs)
        first_image_idxs = [i[0] for i in first_image_idxs if len(i)>0]
        image_evs = [image_evs[i] for i,ev in enumerate(image_evs) if i in first_image_idxs]
        print "N image evs after taking only first image: ", len(image_evs)
        
        # Filter out image-events that were aborted:
        aborted_evs = df.get_events('trial_aborted')
        aborted_idxs = []
        for idx,imev in enumerate(image_events[0:-1]):
            check_abort = [i.value for i in aborted_evs if i.time>imev.time and i.time<image_events[idx+1].time]
            if sum(check_abort)>0:
                aborted_idxs.append(idx)
        print "N aborted images: ", len(aborted_idxs)
        image_evs = [image_evs[i] for i,ev in enumerate(image_evs) if i not in aborted_idxs]
        print "N image evs after removing aborted: ", len(image_evs)
        
        # Find blank ITIs:
        if mask is True:
            iti_evs = [i for i in pixelclock_evs if i.time>tmp_image_evs[0].time and i not in tmp_image_evs]
        else:
#             prevdev = [[i for i,d in enumerate(display_evs) if d.time < t.time][-1] for t in image_evs[1:]]
#             lastdev = [i for i,d in enumerate(display_evs) if d.time > image_evs[-1].time and len(d.value)<num_non_stimuli] # ignore the last "extra" ev (has diff time-stamp) - just wnt 1st non-grating blank
#             iti_evs = [display_evs[i] for i in prevdev]
#             if len(lastdev)>0:
#                 iti_evs.append(display_evs[lastdev[0]])
            #nonstim_evs = [i for i in display_evs if i not in tmp_image_evs]
            
            im_idx = [[t.time for t in display_evs].index(i.time) for i in image_evs]
            iti_evs = []
            for im in im_idx:
                try:
                    next_iti = next(i for i in display_evs[im:] if len(i.value)==(num_non_stimuli-1))
                    iti_evs.append(next_iti)
                except StopIteration:
                    print "No ITI found after this (should be last image_ev).\n"
                    #print display_evs[im]
            print "got iti events"
        
        # Check that we got all the blanks:
#         blanks = [i for i,p in enumerate(pixelclock_evs) if len(p.value)==(num_non_stimuli-1)]
#         mismatches = [i for i,(p,t) in enumerate(zip([pixelclock_evs[x] for x in blanks], iti_evs)) if not p==t]   
#         if len(mismatches)>0:
#             print "Mismatches found in parsing trials...."
#             print mismatches

        # Append a "trial off" at end, if needed:
        if image_evs[-1].time > iti_evs[-1].time: # early-abort
            print "Removing extra image event that has no offset."
            image_evs.pop(-1)                
        tmp_trial_evs = image_evs + iti_evs
        trial_evs = sorted(tmp_trial_evs, key=get_timekey)

    #trialevents.append(tmp_trialevents)
    print "Length of trial epochs: ", len(trial_evs)
    print "Number of trials found: ", len(image_evs)
    
    return image_evs, trial_evs

In [120]:
newstim_evs = df.get_events('new_stimulus')
print [i for i,v in enumerate(newstim_evs[0:-1]) if v.value==1 and newstim_evs[i+1].value==1]

newstim_evs = df.get_events('new_stimulus')
new_stim_evs = [i for i in newstim_evs if i.value==1]
first_image_idxs = []
for idx,newev in enumerate(new_stim_evs[0:-1]):
    possible_image_evs = [i for i,ev in enumerate(image_events) if ev.time>newev.time and ev.time<new_stim_evs[idx+1].time]
    first_image_idxs.append(possible_image_evs)

#first_ims = [i[0] for i in first_image_idxs if len(i)>0]
first_ims = [len(i) for i in first_image_idxs]

print first_ims

[]
[1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

In [85]:
aborted_evs = df.get_events('trial_aborted')
aborted_idxs = []
for idx,imev in enumerate(image_events[0:-1]):
    check_abort = [i.value for i in aborted_evs if i.time>imev.time and i.time<image_events[idx+1].time]
    if sum(check_abort)>0:
        aborted_idxs.append(idx)
print "N aborted images: ", len(aborted_idxs)
print len(image_events)
image_events = [image_events[i] for i,ev in enumerate(image_events) if i not in aborted_idxs]
print len(image_events)

N aborted images:  183
400
217


In [21]:
def get_session_info(df):
    info = dict()
    
    stimon = df.get_events('stim_on_time')
    info['stimulus_duration'] = stimon[-1].value
    itis = df.get_events('ITI_duration')
    info['iti_duration'] = itis[-1].value
    sizes = df.get_events('fullscreen_size')
    info['stimsize'] = sizes[-1].value
    # stimulus types?
    # ntrials?
    portside = df.get_events('A_port')
    info['A_port'] = portside[-1].value
    
    return info

In [22]:
source_dir = '/nas/volume1/behavior/data/mworks-data/head_fixed'
experiment = 'gratings'
version = 'v1'
nanimals = 4

data_dir = os.path.join(source_dir, experiment, version)
tmp_fns = os.listdir(data_dir)
fns = [f for f in tmp_fns if f.endswith('.mwk') and os.path.isfile(os.path.join(data_dir, f))]
print "Found %i MW data files." % len(fns)

Found 20 MW data files.


In [23]:
session_dates = list(set([f[:-4].split('_')[0] for f in fns]))
print "Sessions: ", session_dates

animal_names = list(set([f[:-4].split('_')[1] for f in fns]))
if len(animal_names)>nanimals:
    print "More than %i expected IDs found:"
    for a,name in enumerate(animal_names):
        print a, ": ", name
    discard = raw_input("Enter idx of repeat names:\n")
    if len(discard)>0:
        discard = int(discard)
        print "Removing ID: %s" % animal_names[discard]
        animal_names.pop(discard)

print "Animals: ", animal_names

Sessions:  ['20170806', '20170804', '20170805', '20170802', '20170803']
More than %i expected IDs found:
0 :  Minerva
1 :  Hera
2 :  Athena
3 :  Juno
4 :  Juno2
Enter idx of repeat names:

Animals:  ['Minerva', 'Hera', 'Athena', 'Juno', 'Juno2']


In [24]:
# Create dict of data for all animals and sessions:
datafiles = dict()
for animal in animal_names:
    animal_files = [f for f in fns if animal in f]
    datafiles[animal] = dict((session, dict()) for session in session_dates)
    for session in datafiles[animal].keys():
        datafiles[animal][session]['files'] = [os.path.join(data_dir, f) for f in animal_files if session in f]
datafiles

{'Athena': {'20170802': {'files': ['/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170802_Athena.mwk']},
  '20170803': {'files': ['/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170803_Athena.mwk']},
  '20170804': {'files': ['/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170804_Athena.mwk']},
  '20170805': {'files': ['/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170805_Athena.mwk']},
  '20170806': {'files': ['/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170806_Athena.mwk']}},
 'Hera': {'20170802': {'files': ['/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170802_Hera.mwk']},
  '20170803': {'files': ['/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170803_Hera.mwk']},
  '20170804': {'files': ['/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170804_Hera.mwk']},
  '20170805': {'files': ['/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/201

In [141]:
animal = datafiles.keys()[0]
session = sorted(datafiles[animal].keys())[3]
print session
print datafiles[animal][session]
files = datafiles[animal][session]['files']
f = files[0]
# for f in files:
df, bounds = get_session_bounds(f)
t_boundary = bounds[0]

20170805
{'files': ['/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170805_Minerva.mwk']}


****************************************************************
Parsing file
/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170805_Minerva.mwk... 
Found 1 start events in session.
Bounds:  [[66522679, 4659985413]]
bound ID: 0 4593.462734 sec
****************************************************************


In [142]:
image_events, all_trial_events = get_image_events(df, t_boundary, stimtype='grating')
session_info = get_session_info(df)
print session_info

No pixel clock.
got image events
N new-stimulus events: 686
N image evs after taking only first image:  603
N aborted images:  0
N image evs after removing aborted:  603
got iti events
Length of trial epochs:  1206
Number of trials found:  603
{'A_port': 1, 'stimsize': 100, 'stimulus_duration': 2000, 'iti_duration': 2000}


In [143]:
#print image_events[0]
#print all_trial_events[3]
iti_events = [ev for ev in all_trial_events[1::2]]

print [idx for idx,i in enumerate(iti_events) if len(i.value)>1]
# print all_trial_events[-2]
# print all_trial_events[-1]
# print image_events[-1]

[]


In [144]:
# If no "blank" period after image, experiment ended mid-stimulus, so remove:
if len(image_events)*2 > len(all_trial_events):
    image_events.pop(-1)
print "N trials: ", len(image_events)
print "N trial epochs: ",  len(all_trial_events)

# Filter out any non-outcome trials:
iti_events = [ev for ev in all_trial_events[1::2]]
print "n imagess: ", len(image_events)
print "n itis: ", len(iti_events)

N trials:  603
N trial epochs:  1206
n imagess:  603
n itis:  603


In [145]:
# Parse stimulus info for each trial:
print "Found %i image trials." % len(image_events)
print "Found %i stimulus update events across trials." % len(all_trial_events)

# GET TRIAL INFO FOR DB:
trial = dict((i+1, dict()) for i in range(len(image_events)))
image_events = sorted(image_events, key=get_timekey)
trial_events = sorted(all_trial_events, key=get_timekey)
runstart_time = trial_events[0].time
for trialidx,iev in enumerate(sorted(image_events, key=get_timekey)):
    trialnum = trialidx + 1
    blankidx = trialidx*2 + 1
    trial[trialnum]['start_time_ms'] = round(iev.time/1E3)
    #trial[trialnum]['end_time_ms'] = round((trial_events[blankidx].time + session_info['ITI'])/1E3)
    ori = iev.value[1]['rotation']
    sf = iev.value[1]['frequency'] #round(iev.value[1]['frequency'], 2)
    stimname = 'grating-ori-%i-sf-%0.2f' % (ori, sf)
    stimpos = [iev.value[1]['xoffset'], iev.value[1]['yoffset']]
    stimsize = iev.value[1]['height']
    trial[trialnum]['stimuli'] = {'stimulus': stimname, 'position': stimpos, 'scale': stimsize}
    trial[trialnum]['stim_on_times'] = round((iev.time - runstart_time)/1E3)
    trial[trialnum]['stim_off_times'] = round((trial_events[blankidx].time - runstart_time)/1E3)

print "Trial 1 info:"
print trial[2]

Found 603 image trials.
Found 1206 stimulus update events across trials.
Trial 1 info:
{'stim_off_times': 10033.0, 'stimuli': {'stimulus': 'grating-ori-90-sf-0.06', 'position': [0, 0], 'scale': 100}, 'start_time_ms': 75561.0, 'stim_on_times': 6383.0}


In [146]:
stim_on_durations = [trial[t]['stim_off_times'] - trial[t]['stim_on_times'] for t in trial.keys()]
print "longest: ", [v for v in stim_on_durations].index(max(stim_on_durations))

longest:  512


In [147]:
# Plot histogram of stimulus-ON durations
p1 = figure(title="Stimulus on durations (s)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")

hist, edges = np.histogram(np.array(stim_on_durations)/1E3, density=True, bins=20)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
show(p1)

In [95]:
codec = df.get_codec()

In [148]:
outcome_reached_evs = df.get_events('outcome_reached')
found_outcome_evs = []
for idx,(im,iti) in enumerate(zip(image_events, iti_events)):
    next_outcome = [i for i in outcome_reached_evs if i.time>im.time and i.time<iti.time and i.value==1]
    found_outcome_evs.append(next_outcome)

print [len(i) for i in found_outcome_evs]

[2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [149]:
# Since training flags influence which and how many outcomes are possible on each trial, need to distinguish:
# first response from all responses. For example, an initial "ignore" or "fail" trial can still have a "success"
# if training flag flag_guarantee_reward is on.

outcome_reached_evs = df.get_events('outcome_reached')
first_outcome_idxs = [ix+1 for ix,val in enumerate(np.diff([ev.value for ev in outcome_reached_evs])) if val==1]
first_outcome_evs = [outcome_reached_evs[i] for i in first_outcome_idxs]
first_outcome_evs = sorted([i for i in first_outcome_evs], key=lambda e: e.time)

success_evs = df.get_events('n_successes_session')
fail_evs = df.get_events('n_failures_session')
ignore_evs = df.get_events('n_ignores_session')

result_evs = []
for idx,oev in enumerate(first_outcome_evs):
    if idx==len(first_outcome_evs)-1:  # don't need upper bound on time, since last outcome event:
        tmp_success_ev = [i for i in success_evs if i.time > oev.time]
        tmp_fail_ev = [i for i in fail_evs if i.time > oev.time]
        tmp_ignore_ev = [i for i in ignore_evs if i.time > oev.time]
    else:
        tmp_success_ev = [i for i in success_evs if i.time > oev.time and i.time < first_outcome_evs[idx+1].time]
        tmp_fail_ev = [i for i in fail_evs if i.time > oev.time and i.time < first_outcome_evs[idx+1].time]
        tmp_ignore_ev = [i for i in ignore_evs if i.time > oev.time and i.time < first_outcome_evs[idx+1].time]
    #print [len(tmp_success_ev), len(tmp_fail_ev), len(tmp_ignore_ev)]
    tmp_result_evs = [tmp_success_ev, tmp_fail_ev, tmp_ignore_ev]
    tmp_result_evs = [i[0] for i in tmp_result_evs if len(i)>0] # only take the first occurrence
    result_times = [(idx, i.time) for idx,i in enumerate(tmp_result_evs) if len(i)>0]
    result_idx = [ev for ev in result_times if ev[1]==min([i[1] for i in result_times])][0]
    result_ev = tmp_result_evs[result_idx[0]]
    result_evs.append(result_ev)
    
print len(first_outcome_evs)
print len(result_evs)
print len(image_events)

604
604
603


In [ ]:
def to_trials(stim_display_events, outcome_events, remove_unknown=True,
              duration_multiplier=2, stim_blacklists=None):
    """
    If remove_unknown, any trials where a corresponding outcome_event cannot
    be found will be removed.

    If duration_multiplier is not None, to_trials will check to see if the
    outcome event occured within duration_multiplier * duration microseconds
    of the trial start. If the outcome event occured later, the trial outcome
    will be marked as unknown.
    """
    if (len(outcome_events) == 0) or (len(stim_display_events) == 0):
        return []
    assert hasattr(outcome_events[0], 'name')

    trials = pymworks.events.display.to_stims(stim_display_events, as_dicts=True,
                      blacklist=stim_blacklists)

    if (len(trials) == 0):
        return []

    outcomes = utils.sync(outcome_events, trials,
                          direction=1, mkey=lambda x: x['time'])

    assert len(trials) == len(outcomes)
    unknowns = []
    if duration_multiplier is None:
        dtest = lambda t, o: True
    else:
        dtest = lambda t, o: \
            o.time < (t['time'] + t['duration'] * duration_multiplier)
    for i in xrange(len(trials)):
        if (outcomes[i] is not None) and dtest(trials[i], outcomes[i]):
            trials[i]['outcome'] = outcomes[i].name
        else:
            if remove_unknown:
                unknowns.append(i)
            else:
                trials[i]['outcome'] = 'unknown'

    # remove trials with 'unknown' outcome, in reverse
    for u in unknowns[::-1]:
        del trials[u]

    return trials

In [150]:
print len(image_events)
test_trials = pymworks.events.display.to_stims(image_events, as_dicts=True,
                  blacklist=None)
print len(test_trials)
bad_ims = []
for idx,(im,tr) in enumerate(zip(image_events[0:len(test_trials)], test_trials)):
    if not im.time==tr['time']:
        bad_ims.append(idx)
print bad_ims

print "test trial: ", test_trials[-1]

print "last im: ", image_events[-2]


603
602
[]
test trial:  {'direction': 0, 'start_time': 4634777721, 'time': 4634777721, 'yoffset': 0, 'frequency': 0.06, 'duration': 4716669, 'rotation': 0, 'height': 100, 'speed': 0, 'xoffset': 0, 'name': 'fullscreen_vertical_grating', 'current_phase': -0.0, 'mask': 'rectangle', 'width': 100, 'alpha_multiplier': 1, 'grating': 'square', 'action': 'draw', 'type': 'drifting_grating', 'starting_phase': 0}
last im:  Event[code=7, name=#stimDisplayUpdate, time=4634777721, value=[{'name': 'background_black', 'color_g': 0.0, 'color_b': 0.0, 'action': 'draw', 'type': 'blankscreen', 'color_r': 0.0}, {'direction': 0, 'start_time': 4634777721, 'name': 'fullscreen_vertical_grating', 'type': 'drifting_grating', 'starting_phase': 0, 'mask': 'rectangle', 'yoffset': 0, 'width': 100, 'frequency': 0.06, 'alpha_multiplier': 1, 'grating': 'square', 'action': 'draw', 'rotation': 0, 'height': 100, 'speed': 0, 'current_phase': -0.0, 'xoffset': 0}]]


In [151]:
# Check respomnse times:
response_times = []
# for idx,(im,res) in enumerate(zip(test_trials, result_evs)):
#     response_time = (res.time - im['time'])/1E6
#     response_times.append(response_time)
for idx,(im,res) in enumerate(zip(image_events[0:len(result_evs)], result_evs)):
    response_time = (res.time - im.time)/1E6
    response_times.append(response_time)
    
# Plot histogram of response-times:
p1 = figure(title="Response time from stim onset (s)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")

hist, edges = np.histogram(response_times, density=True, bins=20)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
show(p1)

In [152]:
# Assign FIRST outcome to each trial:
tmp_trial_events = pymworks.events.display.to_trials(image_events, result_evs, duration_multiplier=1.5, remove_unknown=True)
print "N result events: ", len(result_evs)
print "N trial events: ", len(tmp_trial_events)
mismatch = []
for idx,(res,tri) in enumerate(zip(result_evs, tmp_trial_events[0:len(result_evs)])):
    if not res.name==tri['outcome']:
        mismatch.append(idx)
print mismatch

N result events:  604
N trial events:  602
[588]


In [153]:
print [r.name for r in result_evs[39:59]]
print [t['outcome'] for t in tmp_trial_events[39:59]]

result_evs[40].time

['n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_failures_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_failures_session', 'n_ignores_session']
['n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_failures_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_ignores_session', 'n_failures_session', 'n_ignores_session']


405412185

In [151]:
# Tally up each outcome type:
nsuccess= 0; nfail = 0; nignore = 0;
for t in tmp_trial_events:
    if 'successes' in t['outcome']:
        nsuccess += 1
    elif 'fails' in t['outcome']:
        nfail += 1
    elif 'ignores' in t['outcome']:
        nignore += 1

print "N successes: ", nsuccess
print "N fails: ", nfail
print "N ignores: ", nignore

N successes:  180
N fails:  0
N ignores:  34


In [160]:
# Determine state of training flags:
pre_reward = df.get_events('flag_pre_reward')
pre_reward = [i for i in pre_reward if i.time>= t_boundary[0] and i.time<=t_boundary[1]]
print np.diff([i.time for i in pre_reward])/1E6

guarantee_reward = df.get_events('flag_guarantee_reward')
guarantee_reward = [i for i in guarantee_reward if i.time>=t_boundary[0] and i.time<=t_boundary[1]]
print np.diff([i.time for i in guarantee_reward])/1E6
print guarantee_reward

[  19.030678  109.591028]
[]
[Event[code=52, name=flag_guarantee_reward, time=85004365418, value=1]]
